In [ ]:
import pandas as pd
import numpy as np

In [ ]:
clipin= '::21e:5e09:23c:235e'

df = pd.read_csv('s3://hackathon-nerual-network-datasets/' + clipin[2:] + '_Appliance_Flow_Temperature.csv', \
                 parse_dates=[4])

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['hour'] = df['ts'].dt.hour
df['day'] = df['ts'].dt.day
df = df[['hour', 'day', 'value']]
df.head()

In [ ]:
df_norm = df.copy()

In [ ]:
df_norm['value']= MinMaxScaler().fit_transform(df_norm['value'].values.reshape(df_norm.shape[0],1))
df_norm['day']= MinMaxScaler().fit_transform(df_norm['day'].values.reshape(df_norm.shape[0],1))
df_norm['hour']= MinMaxScaler().fit_transform(df_norm['hour'].values.reshape(df_norm.shape[0],1))
df_norm.head()

In [ ]:
df_norm['lag15'] = df_norm['value'].shift(15)
df_norm['lag60'] = df_norm['value'].shift(60)
df_norm = df_norm.dropna().reset_index(drop=True)
df_norm.head()

In [ ]:
df_norm = df_norm[['lag15','lag60','hour','day','value']].rename(columns={'value':'target'})
df_norm.head()

In [ ]:
df_norm.shape[0]

In [ ]:
from io import StringIO
import boto3

csv_buffer = StringIO()
df_norm.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource('s3')

s3_resource.Object('hackathon-nerual-network-datasets', clipin[2:] + '_Appliance_Flow_Temperature_Features_Norm.csv').put(Body=csv_buffer.getvalue())